In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git config --global credential.helper store

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `HF_token` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cach

In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/Creativity/Qwen2.5_7b_16.xlsx'
df = pd.read_excel(file_path, sheet_name ='Sheet1')

In [ ]:
# Load the model and tokenizer
model_name = "Qwen/Qwen2.5-7B-Instruct"
# model_path = '/content/drive/MyDrive/Creativity/Qwen2.5_7b_fine_tuned_model_euclaise_WritingPrompts_full'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
pad_token_id = tokenizer.eos_token_id

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

In [ ]:
def generate_response(prompt, n=1 ,max_length=300):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    attention_mask = (input_ids != pad_token_id).long().to(device)
    responses = []
    for _ in range(n):
        with torch.no_grad():
            output = model.generate(input_ids,
                         attention_mask=attention_mask,
                         max_length=max_length,
                         do_sample=True,
                         top_k=50,
                         top_p=0.9,
                         temperature=1.0,
                         pad_token_id=pad_token_id)
            response = tokenizer.decode(output[0], skip_special_tokens=True)
            response = response[len(prompt):].strip()
        responses.append(response)
    return responses

In [ ]:
print(df.columns)

Index(['Prompts', 'Response_1', 'Response_2', 'Response_3', 'Functionality',
       'Fluency', 'Flexibility', 'Originality', 'Elaboration'],
      dtype='object')


In [ ]:
# Ensure the columns can hold string data by converting them to string type
df['Response_1'] = df['Response_1'].astype(str)
df['Response_2'] = df['Response_2'].astype(str)
df['Response_3'] = df['Response_3'].astype(str)

In [ ]:
# Generate responses for each prompt and add them to the DataFrame
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating Responses"):
    prompt = row['Prompts']
    responses = generate_response(prompt)
    df.at[idx, 'Response_1'] = responses[0]
    responses = generate_response(prompt)
    df.at[idx, 'Response_2'] = responses[0]
    responses = generate_response(prompt)
    df.at[idx, 'Response_3'] = responses[0]

Generating Responses: 100%|██████████| 100/100 [1:26:48<00:00, 52.08s/it]


In [ ]:
# Save the updated DataFrame back to Google Drive
updated_file_path = '/content/drive/MyDrive/Creativity/Qwen2.5_7b_16.xlsx'
df.to_excel(updated_file_path, index=False, engine='openpyxl')